In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import glob

In [5]:
import os
import pickle

In [46]:
for i in range(10):
    if i ==30:
        break
else:
    print('11')

11


In [5]:
all_csvs = glob.glob('data/s_nfs_model_for_ml/*.csv')

In [28]:
df = pd.read_csv(all_csvs[1])

In [29]:
df

,vehicle_id,velocity,changed_line,type,pos_x,pos_y,distance_prev,distance_next,time
0,8863171221134380171530819377170896357,5,False,HDV,0,0,1,10,0
1,137359989859345751986368655129989596218,5,False,AV,11,0,10,11,0
2,33128003915771874161996989168048555981,5,False,HDV_DEF,23,0,11,6,0
3,67924099065562193339702259796390348844,5,False,HDV_DEF,30,0,6,4,0
4,180498720635163394443823915752666694708,5,False,HDV_DEF,35,0,4,7,0
...,...,...,...,...,...,...,...,...,...
98995,179127729625399758694475027611916778911,5,False,AV,453,1,1,21,999
98996,48203085785251238059817942562799395903,5,False,AV,475,1,21,10,999
98997,281188607545759550569287925499156758862,5,False,HDV_DEF,486,1,10,5,999
98998,281595833580348730024900785738880892901,5,False,HDV_DEF,492,1,5,3,999


In [33]:
for col in df.columns:
    if np.any(df[col].isna()):
        print(col)

In [34]:
df.describe()

,velocity,pos_x,pos_y,distance_prev,distance_next,time
count,99000.000000,99000.000000,99000.000000,99000.000000,99000.000000,99000.000000
mean,4.764040,248.475980,0.500404,9.101010,9.101010,499.500000
std,0.529439,144.180014,0.500002,10.356665,10.356665,288.676448
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,5.000000,123.000000,0.000000,3.000000,3.000000,249.750000
50%,5.000000,248.000000,1.000000,6.000000,6.000000,499.500000
75%,5.000000,373.000000,1.000000,11.000000,11.000000,749.250000
max,5.000000,499.000000,1.000000,81.000000,81.000000,999.000000


In [35]:
set(df['type'])

{'AV', 'HDV', 'HDV_DEF'}

In [37]:
def clean_vehicle_ids(df, id0=0):
    dfres = df.copy()
    for vid in set(df['vehicle_id']):
        dfres.loc[dfres['vehicle_id'] == vid, 'vehicle_id'] = id0
        id0 += 1
    return dfres
df = clean_vehicle_ids(df)
df

,vehicle_id,velocity,changed_line,type,pos_x,pos_y,distance_prev,distance_next,time
0,51,5,False,HDV,0,0,1,10,0
1,48,5,False,AV,11,0,10,11,0
2,12,5,False,HDV_DEF,23,0,11,6,0
3,36,5,False,HDV_DEF,30,0,6,4,0
4,2,5,False,HDV_DEF,35,0,4,7,0
...,...,...,...,...,...,...,...,...,...
98995,76,5,False,AV,453,1,1,21,999
98996,16,5,False,AV,475,1,21,10,999
98997,62,5,False,HDV_DEF,486,1,10,5,999
98998,47,5,False,HDV_DEF,492,1,5,3,999


In [38]:
def extract_step(df, step):
    dfs = df[df['time'] == step]
    return dfs

dfs = extract_step(df, 990)
dfs

,vehicle_id,velocity,changed_line,type,pos_x,pos_y,distance_prev,distance_next,time
98010,22,5,False,HDV,4,0,54,8,990
98011,90,5,False,HDV_DEF,13,0,8,1,990
98012,89,5,False,AV,15,0,1,25,990
98013,9,5,False,HDV_DEF,41,0,25,15,990
98014,15,5,False,HDV_DEF,57,0,15,7,990
...,...,...,...,...,...,...,...,...,...
98104,70,3,False,HDV_DEF,462,1,2,10,990
98105,28,3,False,HDV_DEF,473,1,10,5,990
98106,14,5,False,HDV_DEF,479,1,5,8,990
98107,5,5,False,HDV_DEF,488,1,8,4,990


In [43]:
def window_vehicle(dfs, vehicle_id, width_forward, width_back, max_pos = 499, x4test=None):
    vrow = dfs[dfs['vehicle_id'] == vehicle_id]
    x = vrow['pos_x'].values[0]
    y = vrow['pos_y'].values[0]
    if x4test is not None:
        x = x4test
    x1 = x - width_back
    x2 = x + width_forward 
    inds = np.logical_and(dfs['pos_x'] >= x1, dfs['pos_x'] < x2)

    if x1 < 0:
        inds = np.logical_or(inds, dfs['pos_x'] > max_pos + x1 )
    if x2 > max_pos:
        inds = np.logical_or(inds, dfs['pos_x'] < (x2 - max_pos) )
    
    df_window = dfs[inds].copy()

    if x1 < 0:
        df_window.loc[df_window['pos_x'] > max_pos + x1, 'pos_x'] -= max_pos+1
    if x2 > max_pos:
        df_window.loc[df_window['pos_x'] < (x2 - max_pos), 'pos_x'] += max_pos+1
    df_window.loc[:, 'pos_x'] -= x -  width_back
    
    return df_window, x, y

width_forward, width_back = 6, 4
df2, x, y = window_vehicle(dfs, 1, width_forward, width_back)
df2

,vehicle_id,velocity,changed_line,type,pos_x,pos_y,distance_prev,distance_next,time
98024,58,5,False,HDV,4,0,21,6,990
98068,68,4,False,HDV_DEF,2,1,6,1,990
98069,1,2,False,AV,4,1,1,0,990
98070,85,2,False,AV,5,1,0,7,990


In [44]:
def npwindow(dfw, width):
    res = np.zeros((2, 2, width), dtype=np.int8)
    # [тип, скорость]
    for i, (rowi, row) in enumerate(dfw.iterrows()):
        vel = int(row['velocity'])
        changed_line = int(row['changed_line'])  
        stp = row['type']
        tp = {'AV': 1, 'HDV': 2, 'HDV_DEF':3}[stp]
        xi = int(row['pos_x'])
        yi = int(row['pos_y'])
        # print(xi, yi, vel)
        res[0, yi, xi] = tp
        res[1, yi, xi] = (vel + 1) * (-1 if changed_line else 1)
    return res
npwindow(df2, width_forward + width_back)

array([[[0, 0, 0, 0, 2, 0, 0, 0, 0, 0],
        [0, 0, 3, 0, 1, 1, 0, 0, 0, 0]],

       [[0, 0, 0, 0, 6, 0, 0, 0, 0, 0],
        [0, 0, 5, 0, 3, 3, 0, 0, 0, 0]]], dtype=int8)

In [6]:
with open('data/megadb_13_13.bin', 'rb') as f:
    megadb = pickle.load(f)

In [7]:
megadb.keys()

dict_keys(['AV', 'HDV', 'HDV_DEF'])

In [8]:
for k in megadb:
    print(k, len(megadb[k]))

AV 5760
HDV 6712
HDV_DEF 6708


In [9]:
megadb['HDV_DEF'][-6000][0][333]

array([[[ 0,  0,  3,  2,  0,  0,  0,  1,  0,  0,  0,  0,  0,  3,  3,  0,
          0,  0,  0,  0,  0,  1,  0,  1,  0,  0,  0],
        [ 0,  0,  2,  0,  0,  0,  3,  0,  0,  0,  1,  1,  0,  0,  0,  0,
          0,  0,  2,  0,  0,  0,  1,  0,  1,  0,  0]],

       [[ 0,  0,  4,  3,  0,  0,  0,  2,  0,  0,  0,  0,  0,  4,  4,  0,
          0,  0,  0,  0,  0, -5,  0,  4,  0,  0,  0],
        [ 0,  0,  3,  0,  0,  0,  2,  0,  0,  0,  3,  3,  0,  0,  0,  0,
          0,  0,  4,  0,  0,  0,  5,  0,  4,  0,  0]]], dtype=int8)

In [10]:
from numba import jit
from numba.core.errors import NumbaDeprecationWarning, NumbaPendingDeprecationWarning
import warnings

warnings.simplefilter('ignore', category=NumbaDeprecationWarning)
warnings.simplefilter('ignore', category=NumbaPendingDeprecationWarning)

In [11]:
@jit
def get_ind(xs_shape0, i_start, seq_len):
    i = i_start
    forslice = np.arange(i_start, i_start + xs_shape0)
    inds = np.zeros((xs_shape0-seq_len+1, seq_len), dtype=np.int32)
    for i in range(0, xs_shape0-seq_len+1):
        # inds.append(np.arange(i+i_start, i_start+i+seq_len))
        # inds.append(forslice[i:i+seq_len])
        inds[i] = forslice[i:i+seq_len]
    return inds

xs = np.array([1,2,3,4,5,6])
ys = np.array([1,2,3,4,5,6])+10

ind = get_ind(xs.shape[0], 0, 3)
ind

array([[0, 1, 2],
       [1, 2, 3],
       [2, 3, 4],
       [3, 4, 5]])

In [12]:
xs[ind]

array([[1, 2, 3],
       [2, 3, 4],
       [3, 4, 5],
       [4, 5, 6]])

In [13]:
xs = np.array([1,2,3,4,5,6])
ys = np.zeros_like(xs)
xs = xs.reshape(-1,1)
ni = 0
inds = []
Xs = []
Ys = []
for i in range(4):
    xs += 10 * i
    ys[:] = i
    ind = get_ind(xs.shape[0], ni, 4)
    inds.append(ind)
    Xs.append(np.array(xs))
    Ys.append(np.array(ys))
    ni += xs.shape[0]
Xs = np.concatenate(Xs)
Ys = np.concatenate(Ys)
inds = np.vstack(inds)

In [14]:
inds

array([[ 0,  1,  2,  3],
       [ 1,  2,  3,  4],
       [ 2,  3,  4,  5],
       [ 6,  7,  8,  9],
       [ 7,  8,  9, 10],
       [ 8,  9, 10, 11],
       [12, 13, 14, 15],
       [13, 14, 15, 16],
       [14, 15, 16, 17],
       [18, 19, 20, 21],
       [19, 20, 21, 22],
       [20, 21, 22, 23]])

In [15]:
Xs[inds]

array([[[ 1],
        [ 2],
        [ 3],
        [ 4]],

       [[ 2],
        [ 3],
        [ 4],
        [ 5]],

       [[ 3],
        [ 4],
        [ 5],
        [ 6]],

       [[11],
        [12],
        [13],
        [14]],

       [[12],
        [13],
        [14],
        [15]],

       [[13],
        [14],
        [15],
        [16]],

       [[31],
        [32],
        [33],
        [34]],

       [[32],
        [33],
        [34],
        [35]],

       [[33],
        [34],
        [35],
        [36]],

       [[61],
        [62],
        [63],
        [64]],

       [[62],
        [63],
        [64],
        [65]],

       [[63],
        [64],
        [65],
        [66]]])

In [16]:
Ys[inds]

array([[0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [2, 2, 2, 2],
       [2, 2, 2, 2],
       [2, 2, 2, 2],
       [3, 3, 3, 3],
       [3, 3, 3, 3],
       [3, 3, 3, 3]])

In [63]:
np.array( [[[1,2,3,4,5,6],
        [ 11,12,13,14,15,16]]]*3).reshape(-1,12)

array([[ 1,  2,  3,  4,  5,  6, 11, 12, 13, 14, 15, 16],
       [ 1,  2,  3,  4,  5,  6, 11, 12, 13, 14, 15, 16],
       [ 1,  2,  3,  4,  5,  6, 11, 12, 13, 14, 15, 16]])

In [ ]:
import torch as T
import torch.nn as nn

In [106]:
np.hstack([[1,2], [3,4,5]])

array([1, 2, 3, 4, 5])

In [150]:
from collections import Counter

In [151]:

for k in megadb:
    episodes = megadb[k]
    c = []
    for xs, ys in tqdm(episodes):
        xxs = xs[:, 1, :, :]
        shapex = xxs.shape
        # print(xxs.reshape(shapex[0], -1).shape)
        # print(ys.shape)
        rr = np.hstack([ys.reshape(-1, 1), xxs.reshape(shapex[0], -1)])
        c.append(xs[0, 0, :, :][ys[0], xs.shape[-1]//2] - 1)
    print(Counter(c))

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5760/5760 [00:00<00:00, 43638.50it/s]


Counter({0: 5760})


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6712/6712 [00:00<00:00, 44493.18it/s]


Counter({1: 6712})


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6708/6708 [00:00<00:00, 44287.20it/s]

Counter({2: 6708})


In [166]:
xs[:50, 0, :, xs.shape[-1]//2].T

array([[2, 2, 2, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3],
       [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 3, 2, 2,
        2, 2, 2, 0, 0, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 3, 0, 2, 2,
        2, 2, 3, 3, 3, 3]], dtype=int8)

In [168]:
xs[:50, 0, :, xs.shape[-1]//2].T[:,ys[:50]]

array([[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2],
       [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3]], dtype=int8)

In [137]:
len(episodes)

5760

In [144]:
xs, ys = episodes[3000]

In [145]:
xs[0, 0, :, :][ys[0], xs.shape[-1]//2] - 1

0

In [116]:
rr[:, 1:] == xxs.reshape(shapex[0], -1)

array([[ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       ...,
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True]])

In [66]:
def megeres2seq_dataset(megadb, seq_len):
    inds = []
    Xs = []
    Ys = []
    ni = 0
    for k in megadb:
        episodes = megadb[k]
        for xs, ys in tqdm(episodes):
            ind = get_ind(xs.shape[0], ni, seq_len)
            inds.append(ind)
            xxs = xs[:, 1, :, :]
            shapex = xxs.shape
            Xs.append(np.concatenate([ypos, xxs.reshape(shapex[0], -1)]))
            Ys.append(ys)
            ni += xs.shape[0]
    Xs = np.concatenate(Xs)
    Ys = np.concatenate(Ys)
    inds = np.vstack(inds)
    dt = T.float32
    return T.tensor(Xs, dtype=dt), T.tensor(Ys, dtype=dt), T.tensor(inds)
            
Xs, Ys, inds = megeres2seq_dataset(megadb, 13)
Xs.shape, Ys.shape, inds.shape

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6708/6708 [00:00<00:00, 42393.26it/s]


(torch.Size([19180000, 54]),
 torch.Size([19180000]),
 torch.Size([18949840, 13]))

In [100]:
Ys[inds[-9099]]

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [54]:
conv = nn.Conv2d(1, 3, 2)

In [56]:
x = Xs[inds[333]][:, None, :, :]

In [57]:
x.shape

torch.Size([13, 1, 2, 27])

In [59]:
y = conv(x)
y.shape

torch.Size([13, 3, 1, 26])

In [96]:
inp = nn.functional.log_softmax(T.tensor([[0,0,0,99.]]), dim=1)
inp

tensor([[-99., -99., -99.,   0.]])

In [98]:
nn.functional.nll_loss(inp, T.tensor([0]))

tensor(99.)